<a href="https://colab.research.google.com/github/Meenasree16/meena/blob/main/final_msme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Total  Book - MSME - Grand Final - 07_10_2024.pdf to Total  Book - MSME - Grand Final - 07_10_2024 (1).pdf


In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from each page of the PDF.
    Returns a list of dicts: [{page_num, text}, ...]
    """
    doc = fitz.open(pdf_path)
    pages = []

    for page_num in range(len(doc)):
        text = doc[page_num].get_text()
        pages.append({
            "page_num": page_num + 1,
            "text": text.strip()
        })

    return pages

In [ ]:
import os

# Confirm the uploaded filename
print(os.listdir())

# Replace with your exact file name
pdf_path = "Total  Book - MSME - Grand Final - 07_10_2024.pdf"
pages = extract_text_from_pdf(pdf_path)

# Optional: Preview first 500 characters
print("Preview Page 1:\n", pages[0]["text"][:500])

['.config', 'Total  Book - MSME - Grand Final - 07_10_2024.pdf', 'Total  Book - MSME - Grand Final - 07_10_2024 (1).pdf', 'sample_data']
Preview Page 1:
 


In [ ]:
from transformers import AutoTokenizer

# Load tokenizer from your fine-tuned TinyLLaMA model
tokenizer = AutoTokenizer.from_pretrained("Vipplav/tinyllama-finetuned-faq")

In [ ]:
# Combine all page texts into one big string
full_text = "\n".join([page["text"] for page in pages])

In [ ]:
def chunk_text_by_tokens(text, tokenizer, chunk_size=350, overlap=50):
    """
    Splits input text into overlapping chunks of chunk_size tokens with overlap.
    """
    # Tokenize entire text
    tokens = tokenizer.encode(text, add_special_tokens=False)

    chunks = []
    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens)

        chunks.append({
            "chunk_id": len(chunks),
            "text": chunk_text
        })

        # Move start forward by (chunk_size - overlap)
        start += chunk_size - overlap

    return chunks

In [ ]:
text_chunks = chunk_text_by_tokens(full_text, tokenizer, chunk_size=350, overlap=50)

print(f"✅ Generated {len(text_chunks)} chunks (~350 tokens each)")
print("📄 Example chunk:\n", text_chunks[0]["text"][:500])

Token indices sequence length is longer than the specified maximum sequence length for this model (58711 > 2048). Running this sequence through the model will result in indexing errors


✅ Generated 196 chunks (~350 tokens each)
📄 Example chunk:
 

HANDBOOK OF GOVERNMENT 
SCHEMES FOR MSMEs
 
The compendium of schemes for MSMEs has been made to eﬀ ectively communicate 
and create awareness about Government of Telangana and Government of India schemes 
for the development of thriving MSME ecosystem in the state. 
 
Government of Telangana has a comprehensive MSME Policy 2024 where in various 
policy level measures have been enshrined for MSMEs to reduce the cost of doing business, 
help MSMEs achieve scale and improve competitiveness. In o


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "Vipplav/tinyllama-finetuned-faq"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()  # inference mode

Loading adapter weights from Vipplav/tinyllama-finetuned-faq led to unexpected keys not found in the model: model.layers.0.self_attn.q_proj.lora_A.default.weight, model.layers.0.self_attn.q_proj.lora_B.default.weight, model.layers.0.self_attn.v_proj.lora_A.default.weight, model.layers.0.self_attn.v_proj.lora_B.default.weight, model.layers.1.self_attn.q_proj.lora_A.default.weight, model.layers.1.self_attn.q_proj.lora_B.default.weight, model.layers.1.self_attn.v_proj.lora_A.default.weight, model.layers.1.self_attn.v_proj.lora_B.default.weight, model.layers.10.self_attn.q_proj.lora_A.default.weight, model.layers.10.self_attn.q_proj.lora_B.default.weight, model.layers.10.self_attn.v_proj.lora_A.default.weight, model.layers.10.self_attn.v_proj.lora_B.default.weight, model.layers.11.self_attn.q_proj.lora_A.default.weight, model.layers.11.self_attn.q_proj.lora_B.default.weight, model.layers.11.self_attn.v_proj.lora_A.default.weight, model.layers.11.self_attn.v_proj.lora_B.default.weight, mode

LlamaModel(
  (embed_tokens): Embedding(32000, 2048)
  (layers): ModuleList(
    (0-21): 22 x LlamaDecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): lora.Linear(
          (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
          (lora_dropout): ModuleDict(
            (default): Dropout(p=0.1, inplace=False)
          )
          (lora_A): ModuleDict(
            (default): Linear(in_features=2048, out_features=8, bias=False)
          )
          (lora_B): ModuleDict(
            (default): Linear(in_features=8, out_features=2048, bias=False)
          )
          (lora_embedding_A): ParameterDict()
          (lora_embedding_B): ParameterDict()
          (lora_magnitude_vector): ModuleDict()
        )
        (k_proj): Linear(in_features=2048, out_features=256, bias=False)
        (v_proj): lora.Linear(
          (base_layer): Linear(in_features=2048, out_features=256, bias=False)
          (lora_dropout): ModuleDict(
            (default): Drop

In [ ]:
def mean_pooling(hidden_states, attention_mask):
    """
    Applies mean pooling to the hidden states, excluding padding tokens.
    """
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
    sum_hidden = torch.sum(hidden_states * input_mask_expanded, dim=1)
    sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
    return sum_hidden / sum_mask

In [ ]:
# def embed_chunks_with_llama(text_chunks, tokenizer, model):
#     embedded_data = []

#     for chunk in text_chunks:
#         text = chunk["text"]

#         # Tokenize
#         inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

#         with torch.no_grad():
#             outputs = model(**inputs, output_hidden_states=True)

#         # Use last hidden state
#         last_hidden = outputs.last_hidden_state

#         # Mean pooling
#         embedding = mean_pooling(last_hidden, inputs['attention_mask']).squeeze().numpy()

#         # Save with metadata
#         embedded_data.append({
#             "chunk_id": chunk["chunk_id"],
#             "text": chunk["text"],
#             "embedding": embedding
#         })

#     return embedded_data

In [ ]:
import torch

def mean_pooling(hidden_states, attention_mask):
    """
    Applies mean pooling to the hidden states, excluding padding tokens.
    """
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
    sum_hidden = torch.sum(hidden_states * input_mask_expanded, dim=1)
    sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
    return sum_hidden / sum_mask

def embed_chunks_with_llama_batch(text_chunks, tokenizer, model, batch_size=8):
    """
    Embeds each chunk using TinyLLaMA in batches on GPU.
    """
    model = model.to("cuda")  # Move model to GPU
    model.eval()

    embedded_data = []

    for i in range(0, len(text_chunks), batch_size):
        batch = text_chunks[i:i + batch_size]
        texts = [chunk["text"] for chunk in batch]

        # Tokenize in batch
        inputs = tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

        # Forward pass without gradient
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
            last_hidden = outputs.last_hidden_state

        # Mean pooling
        pooled = mean_pooling(last_hidden, inputs["attention_mask"])  # (batch_size, hidden_dim)

        # Store each embedding with its metadata
        for j, chunk in enumerate(batch):
            embedded_data.append({
                "chunk_id": chunk["chunk_id"],
                "text": chunk["text"],
                "embedding": pooled[j].cpu().numpy()
            })

        print(f"✅ Embedded {i + len(batch)}/{len(text_chunks)} chunks")

    return embedded_data

In [ ]:
# 🔁 Generate all embeddings
embedded_data = embed_chunks_with_llama_batch(
    text_chunks=text_chunks,
    tokenizer=tokenizer,
    model=model,
    batch_size=8  # Adjust this if memory issues occur
)

# 👀 Preview output
print(f"\n✅ Total embeddings generated: {len(embedded_data)}")

print("📌 First chunk preview:")
print("📝 Text (first 300 chars):\n", embedded_data[0]["text"][:300])
print("📈 Embedding vector (first 5 dims):", embedded_data[0]["embedding"][:5])

✅ Embedded 8/196 chunks
✅ Embedded 16/196 chunks
✅ Embedded 24/196 chunks
✅ Embedded 32/196 chunks
✅ Embedded 40/196 chunks
✅ Embedded 48/196 chunks
✅ Embedded 56/196 chunks
✅ Embedded 64/196 chunks
✅ Embedded 72/196 chunks
✅ Embedded 80/196 chunks
✅ Embedded 88/196 chunks
✅ Embedded 96/196 chunks
✅ Embedded 104/196 chunks
✅ Embedded 112/196 chunks
✅ Embedded 120/196 chunks
✅ Embedded 128/196 chunks
✅ Embedded 136/196 chunks
✅ Embedded 144/196 chunks
✅ Embedded 152/196 chunks
✅ Embedded 160/196 chunks
✅ Embedded 168/196 chunks
✅ Embedded 176/196 chunks
✅ Embedded 184/196 chunks
✅ Embedded 192/196 chunks
✅ Embedded 196/196 chunks

✅ Total embeddings generated: 196
📌 First chunk preview:
📝 Text (first 300 chars):
 

HANDBOOK OF GOVERNMENT 
SCHEMES FOR MSMEs
 
The compendium of schemes for MSMEs has been made to eﬀ ectively communicate 
and create awareness about Government of Telangana and Government of India schemes 
for the development of thriving MSME ecosystem in the state. 
 
Govern

In [ ]:
!pip install faiss-cpu

In [ ]:
import faiss
import numpy as np

In [ ]:
# Convert all embeddings to a numpy float32 matrix
embedding_matrix = np.array([entry["embedding"] for entry in embedded_data]).astype("float32")

# Normalize for cosine similarity (FAISS uses L2 distance)
faiss.normalize_L2(embedding_matrix)

# Create index
index = faiss.IndexFlatIP(embedding_matrix.shape[1])  # IP = inner product (after normalization = cosine)
index.add(embedding_matrix)

print(f"✅ FAISS index built with {index.ntotal} chunks.")

✅ FAISS index built with 196 chunks.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Vipplav/tinyllama-finetuned-faq"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

In [ ]:
def retrieve_chunks(query, embedded_data, index, tokenizer, model, k=5):
    """
    Encodes query using TinyLLaMA and retrieves top-k similar chunks using FAISS.
    """
    # Tokenize and embed query using TinyLLaMA
    inputs = tokenizer(query, return_tensors="pt", truncation=True, max_length=512).to("cuda")
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        last_hidden = outputs.hidden_states[-1]
        query_embedding = mean_pooling(last_hidden, inputs["attention_mask"]).cpu().numpy()

    # Normalize and search
    faiss.normalize_L2(query_embedding)
    distances, indices = index.search(query_embedding, k)

    # Retrieve chunks
    results = [embedded_data[i] for i in indices[0]]
    return results

In [ ]:
query = "How to apply for PMEGP scheme?"
top_chunks = retrieve_chunks(query, embedded_data, index, tokenizer, model, k=3)

for i, chunk in enumerate(top_chunks):
    print(f"\n🔹 Rank {i+1} (Chunk ID {chunk['chunk_id']}):")
    print(chunk["text"][:400])  # Print first 400 characters of the retrieved chunk


🔹 Rank 1 (Chunk ID 79):
thoroughly, and make sure that you fall under the 
 
eligibility criteria.
• 
In order to apply, you need to create a Login ID, which would be used in further 
 
communications.
• 
After you have created your user account, you can Login and Apply Online for the Scheme. 
Before proceeding to apply online, kindly make sure that you have read the Guidelines 
thoroughly and also checked the List of do

🔹 Rank 2 (Chunk ID 3):
IC)
16-17
15 Scheme for Promoting Innovation, Rural Industry & Entrepreneurship (ASPIRE)
18
16 KHADI GRAMODYOG VIKAS YOJANA - Umbrella Scheme
19-20
17 Revamped Scheme of Fund for Regeneration of Traditional Industries (SFURTI)
21-22
18 MSME SUSTAINABLE (ZED) CERTIFICATION
23-24
19 MSME - Innovative (Incubation, IPR and Design)
25-26
20 MSME Champions Scheme (Erstwhile CLCS-TUS)
27
21 Quality Manuf

🔹 Rank 3 (Chunk ID 4):
PS)
32
26 Scheme For Promotion of Manufacturing of Electronic Components and 
Semi-Conductors (Specs)
33
27 Technology Incuba

In [ ]:
query = "How to apply for PMEGP scheme?"
top_chunks = retrieve_chunks(query, embedded_data, index, tokenizer, model, k=3)

for i, chunk in enumerate(top_chunks):
    print(f"\n🔹 Rank {i+1} (Chunk ID {chunk['chunk_id']}):")
    print(chunk["text"][:400])  # Print first 400 characters of the retrieved chunk


🔹 Rank 1 (Chunk ID 79):
thoroughly, and make sure that you fall under the 
 
eligibility criteria.
• 
In order to apply, you need to create a Login ID, which would be used in further 
 
communications.
• 
After you have created your user account, you can Login and Apply Online for the Scheme. 
Before proceeding to apply online, kindly make sure that you have read the Guidelines 
thoroughly and also checked the List of do

🔹 Rank 2 (Chunk ID 3):
IC)
16-17
15 Scheme for Promoting Innovation, Rural Industry & Entrepreneurship (ASPIRE)
18
16 KHADI GRAMODYOG VIKAS YOJANA - Umbrella Scheme
19-20
17 Revamped Scheme of Fund for Regeneration of Traditional Industries (SFURTI)
21-22
18 MSME SUSTAINABLE (ZED) CERTIFICATION
23-24
19 MSME - Innovative (Incubation, IPR and Design)
25-26
20 MSME Champions Scheme (Erstwhile CLCS-TUS)
27
21 Quality Manuf

🔹 Rank 3 (Chunk ID 4):
PS)
32
26 Scheme For Promotion of Manufacturing of Electronic Components and 
Semi-Conductors (Specs)
33
27 Technology Incuba

In [ ]:
def generate_answer(query, retrieved_chunks, tokenizer, model, max_tokens=150):
    """
    Generates an answer using TinyLLaMA based on retrieved chunk context.
    """
    # Combine top retrieved chunks into context
    context = "\n\n".join([chunk["text"] for chunk in retrieved_chunks])

    # RAG-style prompt
    prompt = f"""Answer the following question based only on the provided context.
Be concise and specific.

Context:
{context}

Question:
{query}

Answer:"""

    # Tokenize and move to GPU
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False  # use greedy decoding
        )

    # Decode output
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the answer part
    return output_text.split("Answer:")[-1].strip()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Vipplav/tinyllama-finetuned-faq"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load tokenizer and the main causal language model (which will also be used for embeddings)
model_name = "Vipplav/tinyllama-finetuned-faq"

# Load the model once as AutoModelForCausalLM and move it to GPU
# This model will be used for both generation and obtaining embeddings.
# We assume AutoModelForCausalLM outputs contain hidden states needed for embedding.
try:
    # Attempt to load with bfloat16 for potential memory savings if GPU supports it
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to("cuda")
except:
    # Fallback to default dtype if bfloat16 loading fails
    model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

model.eval() # Set to evaluation mode

# Modify the retrieve_chunks function to use the loaded model directly
def retrieve_chunks(query, embedded_data, index, tokenizer, model, k=5):
    """
    Encodes query using TinyLLaMA and retrieves top-k similar chunks using FAISS.
    """
    # Tokenize and embed query using the single loaded model
    # Ensure output_hidden_states=True is handled if needed by the model type
    inputs = tokenizer(query, return_tensors="pt", truncation=True, max_length=512).to("cuda")
    with torch.no_grad():
        # Pass output_hidden_states=True to get hidden layers from AutoModelForCausalLM
        outputs = model(**inputs, output_hidden_states=True)
        # Use the last hidden state for embedding
        last_hidden = outputs.hidden_states[-1]
        # The mean_pooling function needs the attention mask as well
        query_embedding = mean_pooling(last_hidden, inputs["attention_mask"]).cpu().numpy()

    # Normalize and search
    faiss.normalize_L2(query_embedding)
    distances, indices = index.search(query_embedding, k)

    # Retrieve chunks
    results = [embedded_data[i] for i in indices[0]]
    return results

# Modify the generate_answer function call to use the loaded model directly
def generate_answer(query, retrieved_chunks, tokenizer, model, max_tokens=150):
    """
    Generates an answer using TinyLLaMA based on retrieved chunk context.
    """
    # Combine top retrieved chunks into context
    context = "\n\n".join([chunk["text"] for chunk in retrieved_chunks])

    # RAG-style prompt
    prompt = f"""Answer the following question based only on the provided context.
Be concise and specific.

Context:
{context}

Question:
{query}

Answer:"""

    # Tokenize and move to GPU
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate using the single loaded model
    with torch.no_grad():
        # Remove the explicit attention_mask argument as it's already in **inputs
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,  # use greedy decoding
            # attention_mask=inputs['attention_mask'] # This line was removed
        )

    # Decode output
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the answer part
    return output_text.split("Answer:")[-1].strip()

# Now call the functions using the single 'model' instance
query = "how to apply for ASPIRE scheme?"

# Step 4: Retrieve relevant chunks using the same model (configured as AutoModelForCausalLM)
retrieved = retrieve_chunks(query, embedded_data, index, tokenizer, model, k=3)

# Step 5: Generate answer using those chunks with the same model
answer = generate_answer(query, retrieved, tokenizer, model)

print("🧠 Bot:", answer)

🧠 Bot: For Application, https://wehub.telangana.gov.in/we-engage/
